In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../py_files')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 6


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-12 08:44:35] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-12 08:45:20] ax.service.ax_client: Generated new trial 96 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 1.0, 'SL_1': 0.00663, 'SL_2': 0.0, 'SL_3': 0.023881, 'LL_1': 0.0, 'LL_2': 1.0, 'LL_3': 0.0, 'P_1': 0.0, 'P_2': 0.794245, 'P_3': 1.0, 'S_1': 0.0, 'S_2': 0.675857, 'S_3': 1.0, 'Water': 0.9043} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs,

7 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000
mean,103.500000,354.0,3.91,67.0,0.238750,0.009562,0.0,0.004000,0.065438,0.247625,0.035937,0.072063,0.142937,0.183688,0.0,0.490625,0.208313,0.300938
std,4.760952,0.0,0.00,0.0,0.075066,0.029509,0.0,0.014478,0.100929,0.060803,0.077341,0.080642,0.102784,0.116630,0.0,0.315586,0.232525,0.228960
min,96.000000,354.0,3.91,67.0,0.137000,0.000000,0.0,0.000000,0.000000,0.122000,0.000000,0.000000,0.000000,0.020000,0.0,0.115000,0.000000,0.000000
25%,99.750000,354.0,3.91,67.0,0.193750,0.000000,0.0,0.000000,0.000000,0.207750,0.000000,0.000000,0.000000,0.094250,0.0,0.271500,0.000000,0.015750
50%,103.500000,354.0,3.91,67.0,0.237000,0.000000,0.0,0.000000,0.000000,0.240500,0.000000,0.056000,0.191500,0.205500,0.0,0.375000,0.105500,0.341500
75%,107.250000,354.0,3.91,67.0,0.265750,0.000000,0.0,0.000000,0.184500,0.286000,0.000000,0.127250,0.205750,0.247500,0.0,0.591250,0.409000,0.467750
max,111.000000,354.0,3.91,67.0,0.439000,0.115000,0.0,0.058000,0.236000,0.364000,0.201000,0.217000,0.292000,0.439000,0.0,1.000000,0.670000,0.612000


In [7]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000
mean,103.500000,0.354,0.391,0.067,0.884868,0.039841,0.0,0.016326,0.312500,0.954789,0.187500,0.275823,0.602353,0.672114,0.0,0.628919,0.455941,0.606743
std,4.760952,0.000,0.000,0.000,0.105421,0.124077,0.0,0.059235,0.478714,0.180843,0.403113,0.290326,0.433701,0.369704,0.0,0.174454,0.487196,0.436752
min,96.000000,0.354,0.391,0.067,0.732715,0.000000,0.0,0.000000,0.000000,0.276626,0.000000,0.000000,0.000000,0.104680,0.0,0.249862,0.000000,0.000000
25%,99.750000,0.354,0.391,0.067,0.788071,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.324844,0.0,0.600752,0.000000,0.023720
50%,103.500000,0.354,0.391,0.067,0.889542,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.254337,0.769735,0.884086,0.0,0.666477,0.246368,0.786591
75%,107.250000,0.354,0.391,0.067,0.987636,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,0.514668,1.000000,1.000000,0.0,0.717626,1.000000,1.000000
max,111.000000,0.354,0.391,0.067,1.000000,0.484689,0.0,0.237330,1.000000,1.000000,1.000000,0.778311,1.000000,1.000000,0.0,0.847026,1.000000,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.00000
mean,103.500000,354.0,3.91,67.0,83.562500,3.346875,0.0,1.400000,22.903125,86.66875,12.578125,25.221875,50.028125,64.290625,0.0,490.625000,208.312500,300.937500,0.00000
std,4.760952,0.0,0.00,0.0,26.273012,10.328237,0.0,5.067149,35.325149,21.28118,27.069423,28.224693,35.974250,40.820608,0.0,315.586201,232.525187,228.959522,0.31305
min,96.000000,354.0,3.91,67.0,47.950000,0.000000,0.0,0.000000,0.000000,42.70000,0.000000,0.000000,0.000000,7.000000,0.0,115.000000,0.000000,0.000000,-0.70000
25%,99.750000,354.0,3.91,67.0,67.812500,0.000000,0.0,0.000000,0.000000,72.71250,0.000000,0.000000,0.000000,32.987500,0.0,271.500000,0.000000,15.750000,-0.08750
50%,103.500000,354.0,3.91,67.0,82.950000,0.000000,0.0,0.000000,0.000000,84.17500,0.000000,19.600000,67.025000,71.925000,0.0,375.000000,105.500000,341.500000,0.00000
75%,107.250000,354.0,3.91,67.0,93.012500,0.000000,0.0,0.000000,64.575000,100.10000,0.000000,44.537500,72.012500,86.625000,0.0,591.250000,409.000000,467.750000,0.35000
max,111.000000,354.0,3.91,67.0,153.650000,40.250000,0.0,20.300000,82.600000,127.40000,70.350000,75.950000,102.200000,153.650000,0.0,1000.000000,670.000000,612.000000,0.35000


In [9]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.3
SL_1       0.1
SL_2       0.0
SL_3       0.0
LL_1       0.4
LL_2       1.4
LL_3       0.2
P_1        0.4
P_2        0.8
P_3        1.0
S_1        0.0
S_2        7.8
S_3        3.3
Water      4.8
Solvent    0.0
dtype: float64

In [10]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [11]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-12 08:52:21] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_6.json`.


# Generate new protocol

In [12]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [13]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [14]:
updated_script

'protocol/iteration_6_OT_2_protocol_.py'

# Perform experiment

# Results

In [16]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,96,354,3.91,67,0.261,0.002,0,0.006,0.000,0.261,0.000,0.000,0.208,0.261,0,0.262,0.388,0.350,7,0
1,97,354,3.91,67,0.181,0.000,0,0.000,0.236,0.236,0.000,0.082,0.211,0.053,0,1.000,0.000,0.000,6,0
2,98,354,3.91,67,0.333,0.000,0,0.000,0.000,0.333,0.000,0.000,0.000,0.333,0,0.362,0.211,0.427,4,0
3,99,354,3.91,67,0.216,0.036,0,0.058,0.000,0.243,0.000,0.000,0.204,0.243,0,0.264,0.402,0.333,7,0
4,100,354,3.91,67,0.245,0.000,0,0.000,0.000,0.302,0.000,0.164,0.191,0.099,0,0.293,0.430,0.277,6,0
5,101,354,3.91,67,0.290,0.000,0,0.000,0.000,0.364,0.000,0.214,0.000,0.132,0,0.425,0.000,0.575,4,0
6,102,354,3.91,67,0.280,0.000,0,0.000,0.000,0.284,0.000,0.217,0.000,0.218,0,0.249,0.334,0.417,5,0
7,103,354,3.91,67,0.439,0.000,0,0.000,0.000,0.122,0.000,0.000,0.000,0.439,0,0.455,0.000,0.545,3,0
8,104,354,3.91,67,0.205,0.000,0,0.000,0.210,0.210,0.000,0.164,0.000,0.210,0,0.115,0.442,0.442,6,0
9,105,354,3.91,67,0.137,0.000,0,0.000,0.182,0.182,0.182,0.040,0.182,0.095,0,1.000,0.000,0.000,7,0


In [17]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,213.982649,0.233609
1,Good,A2,216.875095,0.229261
2,Good,A3,227.115117,0.177695
3,Good,A4,206.686836,0.213478
4,Good,A5,207.958173,0.183084
5,Good,A6,210.366155,0.195140
6,Caution,B1,225.203492,0.215067
7,Caution,B2,237.033216,0.268781
8,Caution,B3,227.317750,0.204535
9,Good,B4,222.480423,0.216309


In [18]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,219.324287,6.900314,0.213521,0.031103,1
1,Formulation 2,208.337055,1.868692,0.197234,0.015305,1
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,215.674568,6.331341,0.226130,0.012024,1
4,Formulation 5,213.753027,10.809642,0.201780,0.025048,1
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,221.993249,1.183270,0.198169,0.016841,1
7,Formulation 8,144.520493,2.256919,0.220073,0.019875,1
8,Formulation 9,197.605274,7.600258,0.188839,0.005599,1
9,Formulation 10,216.524653,3.337142,0.178357,0.014997,1


In [19]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,96,354,3.91,67,0.261,0.002,0,0.006,0.000,0.261,...,0.388,0.350,7,0,Formulation 1,219.324287,6.900314,0.213521,0.031103,1
1,97,354,3.91,67,0.181,0.000,0,0.000,0.236,0.236,...,0.000,0.000,6,0,Formulation 2,208.337055,1.868692,0.197234,0.015305,1
2,98,354,3.91,67,0.333,0.000,0,0.000,0.000,0.333,...,0.211,0.427,4,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,99,354,3.91,67,0.216,0.036,0,0.058,0.000,0.243,...,0.402,0.333,7,0,Formulation 4,215.674568,6.331341,0.226130,0.012024,1
4,100,354,3.91,67,0.245,0.000,0,0.000,0.000,0.302,...,0.430,0.277,6,0,Formulation 5,213.753027,10.809642,0.201780,0.025048,1
5,101,354,3.91,67,0.290,0.000,0,0.000,0.000,0.364,...,0.000,0.575,4,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,102,354,3.91,67,0.280,0.000,0,0.000,0.000,0.284,...,0.334,0.417,5,0,Formulation 7,221.993249,1.183270,0.198169,0.016841,1
7,103,354,3.91,67,0.439,0.000,0,0.000,0.000,0.122,...,0.000,0.545,3,0,Formulation 8,144.520493,2.256919,0.220073,0.019875,1
8,104,354,3.91,67,0.205,0.000,0,0.000,0.210,0.210,...,0.442,0.442,6,0,Formulation 9,197.605274,7.600258,0.188839,0.005599,1
9,105,354,3.91,67,0.137,0.000,0,0.000,0.182,0.182,...,0.000,0.000,7,0,Formulation 10,216.524653,3.337142,0.178357,0.014997,1


In [20]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,96,354,3.91,67,0.261,0.002,0,0.006,0.000,0.261,...,7,0,Formulation 1,219.324287,6.900314,0.213521,0.031103,1,522.0,0
1,97,354,3.91,67,0.181,0.000,0,0.000,0.236,0.236,...,6,0,Formulation 2,208.337055,1.868692,0.197234,0.015305,1,362.0,0
2,98,354,3.91,67,0.333,0.000,0,0.000,0.000,0.333,...,4,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,99,354,3.91,67,0.216,0.036,0,0.058,0.000,0.243,...,7,0,Formulation 4,215.674568,6.331341,0.226130,0.012024,1,432.0,0
4,100,354,3.91,67,0.245,0.000,0,0.000,0.000,0.302,...,6,0,Formulation 5,213.753027,10.809642,0.201780,0.025048,1,490.0,0
5,101,354,3.91,67,0.290,0.000,0,0.000,0.000,0.364,...,4,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,102,354,3.91,67,0.280,0.000,0,0.000,0.000,0.284,...,5,0,Formulation 7,221.993249,1.183270,0.198169,0.016841,1,560.0,0
7,103,354,3.91,67,0.439,0.000,0,0.000,0.000,0.122,...,3,0,Formulation 8,144.520493,2.256919,0.220073,0.019875,1,878.0,0
8,104,354,3.91,67,0.205,0.000,0,0.000,0.210,0.210,...,6,0,Formulation 9,197.605274,7.600258,0.188839,0.005599,1,410.0,0
9,105,354,3.91,67,0.137,0.000,0,0.000,0.182,0.182,...,7,0,Formulation 10,216.524653,3.337142,0.178357,0.014997,1,274.0,0


In [21]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.219324,0.006900,0.261,0.0,0.213521,0.031103,0.583333
1,1,0.208337,0.001869,0.181,0.0,0.197234,0.015305,0.500000
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
3,1,0.215675,0.006331,0.216,0.0,0.226130,0.012024,0.583333
4,1,0.213753,0.010810,0.245,0.0,0.201780,0.025048,0.500000
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
6,1,0.221993,0.001183,0.280,0.0,0.198169,0.016841,0.416667
7,1,0.144520,0.002257,0.439,0.0,0.220073,0.019875,0.250000
8,1,0.197605,0.007600,0.205,0.0,0.188839,0.005599,0.500000
9,1,0.216525,0.003337,0.137,0.0,0.178357,0.014997,0.583333


In [22]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.219324,0.006900,0.261,0.0,0.213521,0.031103,0.583333
1,1,0.208337,0.001869,0.181,0.0,0.197234,0.015305,0.500000
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
3,1,0.215675,0.006331,0.216,0.0,0.226130,0.012024,0.583333
4,1,0.213753,0.010810,0.245,0.0,0.201780,0.025048,0.500000
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
6,1,0.221993,0.001183,0.280,0.0,0.198169,0.016841,0.416667
7,1,0.144520,0.002257,0.439,0.0,0.220073,0.019875,0.250000
8,1,0.197605,0.007600,0.205,0.0,0.188839,0.005599,0.500000
9,1,0.216525,0.003337,0.137,0.0,0.178357,0.014997,0.583333


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,96,354,3.91,67,0.261,0.002,0,0.006,0.000,0.261,...,0.583333,0,Formulation 1,0.219324,0.006900,0.213521,0.031103,1,0.261,0
1,97,354,3.91,67,0.181,0.000,0,0.000,0.236,0.236,...,0.500000,0,Formulation 2,0.208337,0.001869,0.197234,0.015305,1,0.181,0
2,98,354,3.91,67,0.333,0.000,0,0.000,0.000,0.333,...,0.333333,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0.000,0
3,99,354,3.91,67,0.216,0.036,0,0.058,0.000,0.243,...,0.583333,0,Formulation 4,0.215675,0.006331,0.226130,0.012024,1,0.216,0
4,100,354,3.91,67,0.245,0.000,0,0.000,0.000,0.302,...,0.500000,0,Formulation 5,0.213753,0.010810,0.201780,0.025048,1,0.245,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-12 10:44:42] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-12 10:44:42] ax.service.ax_client: Completed trial 96 with data: {'Solu': (0.261, 0), 'Size': (0.219324, 0.0069), 'PDI': (0.213521, 0.031103), 'Complexity': (0.583333, 0)}.


[INFO 02-12 10:44:42] ax.service.ax_client: Completed trial 97 with data: {'Solu': (0.181, 0), 'Size': (0.208337, 0.001869), 'PDI': (0.197234, 0.015305), 'Complexity': (0.5, 0)}.
[INFO 02-12 10:44:42] ax.service.ax_client: Completed trial 98 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.333333, 0)}.
[INFO 02-12 10:44:42] ax.service.ax_client: Completed trial 99 with data: {'Solu': (0.216, 0), 'Size': (0.215675, 0.006331), 'PDI': (0.22613, 0.012024), 'Complexity': (0.583333, 0)}.
[INFO 02-12 10:44:42] ax.service.ax_client: Completed trial 100 with data: {'Solu': (0.245, 0), 'Size': (0.213753, 0.01081), 'PDI': (0.20178, 0.025048), 'Complexity': (0.5, 0)}.
[INFO 02-12 10:44:42] ax.service.ax_client: Completed trial 101 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.333333, 0)}.
[INFO 02-12 10:44:42] ax.service.ax_client: Completed trial 102 with data: {'Solu': (0.28, 0), 'Size': (0.221993, 0.001183), 'PDI': (0.